# Asistente de busqueda de tesis y articulos para la universidad nacional

Para realizar este asistente hicimos uso de las siguientes herramientas


*   LangChain
    > framework para crear aplicaciones usando modelos de lenguajes
*   Chroma
    > Vectorstore para el almacenamiento de embeddings
*   GPT4All
    > Ecosistema para entrenar y desplegar LLM poderosos y personalizados
*   HuggingFace
    > Ecosistema de desarrollo de aplicaciones de aprendizaje automatico


## Instalación de dependencias

In [1]:
!pip -q install huggingface chromadb transformers langchain pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


## Carga de datos en un Dataframe
La información proveniente del proceso de scrapping será almacenada dentro de un dataframe de pandas para su correcto procesamiento

In [2]:
#Correr si les falla la celda para usar pandas
import sys
!{sys.executable} -m pip -q install openpyxl

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
import pandas as pd
df = pd.read_excel('./datos.xlsx')
df

,Link,Title,Authors,Locations,context
0,https://repositorio.unal.edu.co/handle/unal/8763,A comparison between expert systems and autono...,"Gutiérrez, Sergio Armando; Branch Bedoya, Jho...",Universidad Nacional de Colombia Sede Medellí...,"Abstratc. In this article, a revision of liter..."
1,https://repositorio.unal.edu.co/handle/unal/61999,A Contribution to Semi-Automatic Segmentation ...,"Tamayo Quintero, Juan David",Universidad Nacional de Colombia Sede Manizal...,The increase the emerging technologies and the...
2,https://repositorio.unal.edu.co/handle/unal/53687,A criteria based function for reconstructing l...,"Ospina Álvarez, Edison Camilo",Universidad Nacional de Colombia Sede Medellí...,Abstract: Mobile applications equipped with Gl...
3,https://repositorio.unal.edu.co/handle/unal/52152,A formalization for mapping discourses from bu...,"Manrique Losada, Bell",Universidad Nacional de Colombia Sede Medellí...,Requirements elicitation for software engineer...
4,https://repositorio.unal.edu.co/handle/unal/59316,A Framework for Evaluating Maintainability and...,"Alvarez Eraso, Danny Alejandro",Universidad Nacional de Colombia Sede Medellí...,Web Application Frameworks (WAF) and Object-Re...
...,...,...,...,...,...
821,https://repositorio.unal.edu.co/handle/unal/61994,Validación del programa imagej para cuantifica...,"Huerfano Correa, Maximiliano",Universidad Nacional de Colombia Sede Bogotá ...,"Dentro de la práctica de medicina nuclear, la ..."
822,https://repositorio.unal.edu.co/handle/unal/69908,Variable capacity utilization in cycles of ele...,"Castañeda Acevedo, Jaime Andrés",Universidad Nacional de Colombia Sede Medellí...,Este trabajo describe un experimento de labora...
823,https://repositorio.unal.edu.co/handle/unal/75198,Video and Image Processing based on Kernel Rep...,"Molina Giraldo, Santiago",Universidad Nacional de Colombia Sede Manizal...,En este trabajo se proponen diferentes esquema...
824,https://repositorio.unal.edu.co/handle/unal/59740,Video explicativo para la enseñanza y aprendiz...,"Moreno Rojas, Edwin Mauricio",Universidad Nacional de Colombia Sede Bogotá ...,El video explicativo que se ha desarrollado co...


## Carga del modelo LLM usando Langchain

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
local_path = './models/ggml-gpt4all-j-v1.3-groovy.bin'  # replace with your desired local file path

In [11]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, backend="gptj",callbacks=callbacks, verbose=True)

Found model file at  ./models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


## Importe y splitting de datos
Ya que la información se encuentra en un dataframe de pandas vamos a hacer uso de los siguientes Loaders y splitters:

*  DataFrameLoader
*  RecursiveCharacterTextSplitter

In [12]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap  = 25,
    length_function = len,
)

loader = DataFrameLoader(df, page_content_column="context")
texts = text_splitter.split_documents(loader.load())

## Generación de Embeddings
Para generar los embeddings que serán almacenados vamos a hacer uso del siguiente modelo de la libreria de HuggingFace
*  all-MiniLM-L6-v2


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Vectorstore con Chroma

Para el uso de vectorstore usamos chroma y proporcionamos un path para que se persista la información de los embeddings

In [14]:

from langchain.vectorstores import Chroma

# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
db = Chroma.from_documents(documents=texts, embedding = embeddings, persist_directory=persist_directory)
db.persist()
db = None

### Generación de Chain de respuesta

usando la db de chroma como retriever podemos vincular el modelo LLM por medio de una cadena para poder hacer preguntas

In [15]:
from langchain.chains import RetrievalQA
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k":5})

### Generación de cadena
En esta generación de cadena pasamos por parametro que nos retorne los documentos referentes a la pregunta entregada de esta forma podemos vincular articulos o tesis referentes a la pregunta proporcionada

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)



### Query a consultar al modelo

In [17]:
query = "short article about video data analysis"
result = qa({"query": query})

 Video Data Analysis (VDA) is a complex field that requires extensive knowledge of digital signal processing, image and computer vision technologies for the purposeful collection or acquisition and manipulation of large amounts of multimedia content over time periods with low latency. To accomplish this goal one must have an understanding in many topics such as data types used by video applications like RGB format vs YUV420P; frame rate conversion from FPS to NTP, H264/MPEG4 etc., audio processing techniques & codecs for converting uncompressed or compressed formats into digital form and MPEG-2 AAC encoder /decoders. To be able also have a basic knowledge of programming languages such as C++, Java, Python that are used in VDA workflows like real time video analysis/processing with OpenCV library, computer vision libraries (e.g., TensorFlow), machine learning frameworks for identifying patterns and making predictions based on the large amount multimedia data inputed over years.
 Video D

### Obtención de articulos relacionados en caso de existir

In [40]:

def related_docs(result):
    count_docs = {}
    info_docs = {}
    docs = result['source_documents']
    if(len(docs) == 0):
        print('Sorry there is not articles related with your query')
    for i in range(len(docs)):
        count_docs[docs[i].metadata['Link']] = 0
        info_docs[docs[i].metadata['Link']] = docs[i].metadata['Title']
    
    for i in range(len(docs)):
        count_docs[docs[i].metadata['Link']] = count_docs[docs[i].metadata['Link']] + 1
    
    print('please check the following articles in order of importance based on your query:')
    for doc in sorted(count_docs)[::-1]:
        print("link: ", doc, " Title: ", info_docs[doc])

related_docs(result)

please check the following articles in order of importance based on your query:
link:  https://repositorio.unal.edu.co/handle/unal/56549  Title:  A Kernel framework to support video data analysis
link:  https://repositorio.unal.edu.co/handle/unal/21126  Title:  Guía para el desarrollo de aplicaciones interactivas en TDT para Colombia
